Preparação dos arquivos

In [1]:
from modules.utilities import prepare_data_filesystem

prepare_data_filesystem()

Etapa Landing (Extraindo a lista de IDs)

In [4]:
from modules.etl import Landing

Landing.full_process(database='nucleotide',term='Chikungunya',retmax=1000,datetype="pdat",reldate=60)

Etapa bronze (Extraindo as informações dos IDs do XML salvo pela etapa Landing)

In [5]:
from modules.etl import Bronze

Bronze.full_process('nucleotide',rettype='gbc',retmode='xml')

Etapa Silver (Das informações dos IDs retirar o que é interessante e jogar em um formato de arquivo mais adequado para manipulação) -> Em testes ainda

In [1]:
import os
import xmltodict
import pandas as pd
from datetime import datetime
from modules.pubmed import extract_pmcid_from_list
from modules.utilities import extract_pubmed_list, extract_country, extract_collection_date

bronze_path = 'data/bronze'
lista_de_xmls = [os.path.join(bronze_path,file) for file in os.listdir(bronze_path)]

full_df = pd.DataFrame()
for xml in lista_de_xmls:
    with open(xml,'r') as xml_file:
        xml_data = xml_file.read()
    full_df = pd.concat([full_df,pd.DataFrame(xmltodict.parse(xml_data).get('INSDSet').get('INSDSeq'))],ignore_index=True)

In [2]:
# Filtrando colunas
full_df = full_df[['INSDSeq_locus','INSDSeq_length','INSDSeq_update-date','INSDSeq_create-date','INSDSeq_references','INSDSeq_feature-table','INSDSeq_sequence']]

# Renomeando colunas
colunas_a_serem_renomeadas = {
    'INSDSeq_locus': 'locus',
    'INSDSeq_length':'length',
    'INSDSeq_update-date': 'update_date',
    'INSDSeq_create-date': 'create_date',
    'INSDSeq_sequence':'sequence'
    }
full_df = full_df.rename(columns=colunas_a_serem_renomeadas) 

# Alterar tipo das colunas
colunas_a_serem_convertidas = {
    'length': 'int64',
    'update_date':'datetime64[ns]',
    'create_date':'datetime64[ns]'
    }
full_df = full_df.astype(colunas_a_serem_convertidas)

# Extraindo os pubmeds
column = 'INSDSeq_references'
full_df['pubmed'] = full_df[column].apply(lambda x: extract_pubmed_list(cell=x))
full_df.drop(columns=column,inplace=True)

# Extraindo os countrys e data de coleta
column = 'INSDSeq_feature-table'
full_df['country'] = full_df[column].apply(lambda x: extract_country(cell=x)) 
full_df['collection_date'] = full_df['INSDSeq_feature-table'].apply(lambda x: extract_collection_date(cell=x))
full_df.drop(columns=column, inplace=True)

# Explodindo a coluna pubmed
full_df = full_df.explode(column='pubmed')

# Extrai os PMCID para todos os números Pubmed em paralelo
list_of_pubmeds = full_df['pubmed'].drop_duplicates().dropna().tolist()
pmcid_results = extract_pmcid_from_list(list_of_pubmed=list_of_pubmeds, number_of_webscrappers=4)

# Cria um dicionário a partir dos resultados
pmcid_dict = {temp_dict['pubmed_accession_number']: temp_dict['pmcid'] for temp_dict in pmcid_results}

# Mapeia os PMCID de volta para o DataFrame
full_df['pmcid'] = full_df['pubmed'].map(pmcid_dict)